In [1]:
from transformers import AutoModelForCausalLM

In [2]:
model = AutoModelForCausalLM.from_pretrained("/data/yimin/models/base/meta-llama/Meta-Llama-3-8B", device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
model.type

<bound method Module.type of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): L

In [4]:
model.model.layers[0].self_attn.rotary_emb.__dict__

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('inv_freq',
               tensor([1.0000e+00, 8.1462e-01, 6.6360e-01, 5.4058e-01, 4.4037e-01, 3.5873e-01,
                       2.9223e-01, 2.3805e-01, 1.9392e-01, 1.5797e-01, 1.2869e-01, 1.0483e-01,
                       8.5397e-02, 6.9566e-02, 5.6670e-02, 4.6164e-02, 3.7606e-02, 3.0635e-02,
                       2.4955e-02, 2.0329e-02, 1.6560e-02, 1.3490e-02, 1.0990e-02, 8.9523e-03,
                       7.2927e-03, 5.9407e-03, 4.8394e-03, 3.9423e-03, 3.2114e-03, 2.6161e-03,
                       2.1311e-03, 1.7360e-03, 1.4142e-03, 1.1520e-03, 9.3847e-04, 7.6450e-04,
                       6.2277e-04, 5.0732e-04, 4.1327e-04, 3.3666e-04, 2.7425e-04, 2.2341e-04,
                       1.8199e-04, 1.4825e-04, 1.2077e-04, 9.8381e-05, 8.0143e-05, 6.5286e-05,
                       5.3183e-05, 4.3324e-05, 3.5292e-05, 2.8750e-05, 2.3420e-05, 1.9078e-05,
                       1.5542e-05, 1.2660e-05, 1.0313e-

In [5]:
embedding = model.model.embed_tokens

In [6]:
rope = model.model.layers[0].self_attn.rotary_emb

In [7]:
type(embedding)

torch.nn.modules.sparse.Embedding

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/data/yimin/models/base/meta-llama/Meta-Llama-3-8B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
tokenizer.pad_token = tokenizer.eos_token
input = tokenizer(['Hello, how are you today?', 'I am good'], padding=True, return_tensors="pt").to('cuda')

In [16]:
input

{'input_ids': tensor([[  9906,     11,   1268,    527,    499,   3432,     30],
        [    40,   1097,   1695, 128001, 128001, 128001, 128001]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0]], device='cuda:0')}

In [17]:
input_ids = input['input_ids']
input_mask = input['attention_mask']

In [18]:
embedding_output = embedding(input_ids)
embedding_output

tensor([[[ 1.9287e-02, -8.3008e-03,  6.1951e-03,  ...,  2.6398e-03,
          -3.2196e-03, -4.5586e-04],
         [-2.0599e-04, -2.6703e-04,  1.7643e-05,  ...,  1.3638e-04,
           1.1015e-04,  1.6022e-04],
         [ 3.5286e-04, -4.7302e-03,  5.6763e-03,  ..., -8.6670e-03,
          -5.2185e-03,  7.2937e-03],
         ...,
         [-6.5613e-03, -4.8828e-03, -2.4567e-03,  ..., -1.8005e-03,
           6.5613e-04,  1.0443e-04],
         [-7.3547e-03, -8.9111e-03,  3.2959e-03,  ..., -5.8899e-03,
           1.9531e-03,  2.7771e-03],
         [-4.9438e-03, -1.6098e-03,  6.4087e-03,  ...,  1.9989e-03,
          -1.0147e-03, -4.8523e-03]],

        [[ 7.0496e-03, -3.4904e-04, -6.9275e-03,  ..., -4.9133e-03,
           6.1646e-03, -1.0300e-03],
         [ 9.8877e-03, -1.4343e-03, -1.7090e-03,  ..., -6.4087e-03,
          -9.3842e-04,  1.6022e-03],
         [-4.6692e-03, -1.0925e-02, -2.9755e-03,  ..., -9.5215e-03,
          -8.2397e-03,  7.3547e-03],
         ...,
         [-8.3542e-04,  1

In [19]:
embedding_output.squeeze().shape

torch.Size([2, 7, 4096])

In [22]:
output = model.model.embed_tokens(input_ids)
output.shape

torch.Size([2, 7, 4096])

In [11]:
input_ids

tensor([[9906,   11, 1268,  527,  499, 3432,   30]], device='cuda:0')

In [25]:
import torch
## 保存模型
torch.save(model.model.embed_tokens.state_dict(), 'embedding.pth')

In [26]:
## 读取模型
# state_dict = torch.load('embedding.pth')
# embedding = torch.nn.modules.sparse.Embedding(128256, 4096).to('cuda')
# embedding.load_state_dict(state_dict)
state_dict = torch.load('embedding.pth')
embedding = torch.nn.modules.sparse.Embedding(state_dict['weight'].shape[0], state_dict['weight'].shape[1]).to('cuda')
embedding.load_state_dict(state_dict)

<All keys matched successfully>

In [24]:
state_dict['model']['weight'].shape[1]

4096

In [86]:
tokenizer.vocab_size

128000

In [87]:
type(model.model.embed_tokens)

torch.nn.modules.sparse.Embedding

In [88]:
type(embedding)

torch.nn.modules.sparse.Embedding

In [89]:
embedding.load_state_dict(state_dict)

<All keys matched successfully>

In [94]:
output = embedding(input_ids)

In [96]:
output.shape

torch.Size([1, 7, 4096])

In [97]:
model.model.embed_tokens.state_dict()

OrderedDict([('weight',
              tensor([[ 1.3733e-03,  5.0964e-03, -3.0365e-03,  ...,  2.2888e-03,
                       -1.9531e-03, -1.7166e-05],
                      [-2.7313e-03,  1.9379e-03, -1.3733e-03,  ..., -5.1498e-05,
                       -1.3962e-03, -1.9836e-03],
                      [ 9.5367e-04, -1.3367e-02,  4.1771e-04,  ...,  2.5940e-03,
                        7.0496e-03,  4.1809e-03],
                      ...,
                      [ 1.8715e-23,  3.2699e-24,  1.8198e-23,  ...,  5.3767e-23,
                       -2.2360e-24, -1.9852e-23],
                      [ 1.9335e-23, -1.8612e-24, -1.8818e-23,  ...,  2.3368e-23,
                        7.3412e-24, -3.1226e-23],
                      [-7.4860e-23, -6.3693e-23,  5.5059e-24,  ...,  4.9631e-24,
                       -5.4594e-23, -2.2877e-24]], device='cuda:0'))])

In [101]:
model.model.embed_tokens.num_embeddings

128256

In [27]:
print(embedding)

Embedding(128256, 4096)


In [29]:
model.model.embed_tokens.__dict__

{'training': False,
 '_parameters': OrderedDict([('weight',
               Parameter containing:
               tensor([[ 1.3733e-03,  5.0964e-03, -3.0365e-03,  ...,  2.2888e-03,
                        -1.9531e-03, -1.7166e-05],
                       [-2.7313e-03,  1.9379e-03, -1.3733e-03,  ..., -5.1498e-05,
                        -1.3962e-03, -1.9836e-03],
                       [ 9.5367e-04, -1.3367e-02,  4.1771e-04,  ...,  2.5940e-03,
                         7.0496e-03,  4.1809e-03],
                       ...,
                       [ 1.8715e-23,  3.2699e-24,  1.8198e-23,  ...,  5.3767e-23,
                        -2.2360e-24, -1.9852e-23],
                       [ 1.9335e-23, -1.8612e-24, -1.8818e-23,  ...,  2.3368e-23,
                         7.3412e-24, -3.1226e-23],
                       [-7.4860e-23, -6.3693e-23,  5.5059e-24,  ...,  4.9631e-24,
                        -5.4594e-23, -2.2877e-24]], device='cuda:0', requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_